In [51]:
import glob
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output, display

file_names = [_.split('/')[2] for _ in glob.glob('../wiki/*')]

def opener(path):
    with open(path, 'r', encoding='utf-8') as file:
        output = file.read()
    return output

print(file_names)

['Gastrotrich', 'Firefox', 'Torres_Garcia_Museum', 'Sulawesi', 'Oxford_English_Dictionary', 'MEDLINE', 'Nanotronics_Imaging', 'Pharaoh', 'Selenium', 'Buddhism_and_psychology', 'Labial_consonant', 'UN', 'Echinozoa', 'University_of_Chicago', 'Trigeminal_motor_nucleus', 'World_Kendo_Championship', '25th_Amendment_to_the_United_States_Constitution', 'Charlie_Gillett', 'Otto_Struve', 'Orthonectida', 'Basque_language', 'Permian', 'List_of_epidemics', 'Urban_metabolism', 'Team_sport', 'Greek_Dark_Ages', 'Hominina', 'Araucanian_languages', 'Binyamina_train_station_suicide_bombing', 'Google_Calendar', 'Go_handicaps', 'Seymour_Martin_Lipset', 'Bacho_Kiro_cave', 'Amygdala', 'Drug_resistance', 'Method_of_least_squares', 'State_space_search', 'Bayes_factors', 'Software_configuration_management', 'Hot_tapping', 'Forex_fixing', 'Science_education', 'Artificial_castling', 'Abducens_nerve', 'Interpreted_language', 'Optical_Gravitational_Lensing_Experiment', 'Illinois', 'Kananga', 'Type_physicalism', 'S

In [2]:
def parse(path_to_file):
    text = opener(path_to_file)
    soup = BeautifulSoup(text, 'lxml').find('div', id='bodyContent')
    print(soup.find_all('a'))

parse(f'../wiki/{file_names[0]}')

[<a href="#mw-head">navigation</a>, <a href="#p-search">search</a>, <a class="image" href="/wiki/File:Gastrotrich.jpg"><img alt="Gastrotrich.jpg" data-file-height="1003" data-file-width="964" height="229" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Gastrotrich.jpg/220px-Gastrotrich.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Gastrotrich.jpg/330px-Gastrotrich.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Gastrotrich.jpg/440px-Gastrotrich.jpg 2x" width="220"/></a>, <a href="/wiki/Taxonomy_(biology)" title="Taxonomy (biology)">Scientific classification</a>, <a href="/wiki/Animal" title="Animal">Animalia</a>, <a href="/wiki/Platyzoa" title="Platyzoa">Platyzoa</a>, <a href="#cite_note-WoRMS-1">[1]</a>, <a href="/wiki/Chaetonotida" title="Chaetonotida">Chaetonotida</a>, <a href="/wiki/Macrodasyida" title="Macrodasyida">Macrodasyida</a>, <a href="/wiki/Phylum" title="Phylum">phylum</a>, <a href="/wiki/Common_name" title="Common name">commonly<

In [78]:
graph = dict()
def create_graph_dict_node(path_to_file, name):
    pattern = re.compile("/wiki/([\w()]+)")
    link_matched = list()
    text = opener(path_to_file)
    soup = BeautifulSoup(text, 'lxml').find('div', id='bodyContent')
    for _ in soup.find_all('a', href=True):
        _name_ = _['href'].split('/')[-1]
        if (_name_ in file_names) and (_name_ != name):
            try:
                link_matched.append(pattern.findall(_['href'])[0])
            except IndexError:
                pass
    graph.update({f"{name}": list(set(link_matched))})


for i, obj in enumerate(file_names):
    clear_output(wait=True)
    display(str(round(i/len(file_names)*100, 2)) + '%')
    create_graph_dict_node(f'../wiki/{obj}', obj)
graph

'99.8%'

{'Gastrotrich': ['Gymnolaemata',
  'Orthonectida',
  'Rouphozoa',
  'Brachiozoa',
  'Priapulida',
  'Pterobranchia',
  'Echinozoa',
  'Ganglion',
  'Xenoturbella',
  'Echinoderm'],
 'Firefox': ['Web_browser_engine',
  'IBM',
  'HotJava',
  'History_of_the_web_browser',
  'Blacklist',
  'IBrowse',
  'Amaya_(web_editor)',
  'Android_Ice_Cream_Sandwich',
  'Library_of_Congress_Control_Number',
  'Mozilla_application_framework'],
 'Torres_Garcia_Museum': ['Uruguay'],
 'Sulawesi': ['Central_Java', 'Homo_erectus', 'Dutch_East_Indies'],
 'Oxford_English_Dictionary': ['IBM',
  'Gaston_Gonnet',
  'Library_of_Congress_Control_Number',
  'Victorian_morals'],
 'MEDLINE': ['Bioengineering'],
 'Nanotronics_Imaging': ['Artificial_intelligence',
  'Peter_Thiel',
  'Materials_science'],
 'Pharaoh': ['Snaaib', 'Atef'],
 'Selenium': ['Library_of_Congress_Control_Number', 'Moon'],
 'Buddhism_and_psychology': ['Mahayana',
  'Vasubandhu',
  'Buddhism_in_the_United_Kingdom',
  'Mental_illness'],
 'Labial_con

In [80]:
def shortest_way(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if not start in graph.keys():
        return None
    shortest = None
    for node in graph[start]:
        if node not in path:
            newpath = shortest_way(graph, node, end, path)
            if newpath:
                if not shortest or len(newpath) < len(shortest):
                    shortest = newpath
    return shortest


shortest_way(graph=graph, start='The_New_York_Times', end='Stone_Age')


KeyboardInterrupt: 

In [82]:
graph2 = {'A': ['B', 'C'],
             'B': ['C', 'D'],
             'C': ['D'],
             'D': ['C'],
             'E': ['F'],
             'F': ['C']}

shortest_way(graph2, 'A', 'D')

['A', 'B', 'D']

In [86]:
def dfs(adj, v, parent, order):
    if not parent:
        parent[v] = None
    # checking neighbours of v
    for n in adj[v]:
        if n not in parent:
            parent[n] = v
            dfs(adj, n, parent, order)

    # we're done visiting a node only when we're done visiting
    # all of its descendents first
    order.append(v)


def topological_sort(adj):
    parent = {}
    order = []
    for v in adj.keys():
        if v not in parent:
            parent[v] = None
            dfs(adj, v, parent, order)

    return list(reversed(order))

def dag_shortest_path(adj, source, dest):
    order = topological_sort(adj)
    parent = {source: None}
    d = {source: 0}

    for u in order:
        if u not in d: continue  # get to the source node
        if u == dest: break
        for v, weight in adj[u]:
            if v not in d or d[v] > d[u] + weight:
                d[v] = d[u] + weight
                parent[v] = u

    return parent, d

dag_shortest_path(graph, 'The_New_York_Times', 'Stone_Age')

KeyError: 'Template'

In [94]:
from collections import deque
def find_all_paths(graph, start, end, path=[]):
    dist = {start: [start]}
    q = deque(start)
    at = q
    print(at)
    for next in graph[at]:
        if next not in dist:
            dist[next] = [dist[at], next]
            q.append(next)
    return dist.get(end)

find_all_paths(graph, 'The_New_York_Times', 'Stone_Age')

deque(['T', 'h', 'e', '_', 'N', 'e', 'w', '_', 'Y', 'o', 'r', 'k', '_', 'T', 'i', 'm', 'e', 's'])


TypeError: unhashable type: 'collections.deque'

In [62]:
'''     FASTER
text = opener('../wiki/Echinozoa')
soup = BeautifulSoup(text, 'lxml').find('div', id='bodyContent')
for _ in soup.find_all('a', href=re.compile("/wiki/([\w()]+)")):
    print(_['href'].split('/')[-1])
'''

File:Riccio_Melone_a_Capo_Caccia_adventurediving.it.jpg
Sea_urchin
Echinus_melo
Sardinia
Taxonomy_(biology)
Echinozoa
Animal
Echinoderm
Echinoidea
Holothuroidea
Ophiocistioidea
Helicoplacoidea
Arkarua
Echinoderm
Pinnate
List_of_echinodermata_orders
Help:Taxon_identifiers
Wikidata
Q2698547
Animal_Diversity_Web
Integrated_Taxonomic_Information_System
National_Center_for_Biotechnology_Information
World_Register_of_Marine_Species
File:NSRW_Sea_Urchins_and_Star_Fish.jpg
Echinoderm
Wikipedia:Stub
Template:Echinoderm-stub
Template_talk:Echinoderm-stub
Help:Category
Category:Echinozoa
Category:Animal_subphyla
Category:Echinoderm_stubs
Category:Articles_with_%27species%27_microformats
Category:Taxonbar_templates_without_from_parameter
Category:All_stub_articles


Following is Breadth First Traversal (starting from vertex 2)


TypeError: can only concatenate str (not "int") to str

In [129]:
from bs4 import BeautifulSoup
from collections import deque
import re
import os


def build_tree(start, end, path):
    link_re = re.compile(r"(?<=/wiki/)[\w()]+")
    files = dict.fromkeys(os.listdir(path))

    for file in files:
        with open(path+file, 'r') as f:
            files[file] = []
            res = link_re.findall(f.read())
            for name in res:
                if name in files and name not in files[file] and name != file:
                    files[file].append(name)

    search_queue = deque()
    search_queue += files[start]
    file_names = {start: None}

    while search_queue:
        parent = search_queue.popleft()
        for file in files[parent]:
            if file not in file_names:
                search_queue += files[parent]
                file_names[file] = parent

    return file_names


def build_bridge(start, end, path):
    files = build_tree(start, end, path)
    bridge = [end]
    # TODO Добавить нужные страницы в bridge
    file = end
    while file is not start:
        file = files[file]
        if file is None:
            break
        else:
            bridge.append(file)

    return list(reversed(bridge))

build_bridge('The_New_York_Times', 'Stone_Age', '../wiki/')


KeyboardInterrupt: 